In [35]:
import math
from collections import defaultdict
from collections import Counter
import pandas as pd

In [36]:
def create_index(documents):
    # Create a dict to store the term freqs
    index = defaultdict(dict)

    # Create a dict to store the document lengths
    doc_len = {}

    # Create a set to store the unique terms
    terms = set()

    # Loop over the documents
    for i, document in enumerate(documents):

        # Loop over the terms in the document
        for term in document:

            # Change the term frequency
            index[term][i] = index[term].get(i, 0) + 1

            # Add the term to the set
            terms.add(term)

        # Store the length of the document
        doc_len[i] = len(document)

    # Compute the average document length
    avgdl = sum(doc_len.values()) / len(doc_len)

    # Compute the idf for each term
    idf = {}

    #print(terms)

    for term in terms:
        df = len(index[term])
        idf[term] = math.log(len(documents)/df)


    return index, doc_len, avgdl, idf


In [46]:
collection = pd.DataFrame(pd.read_csv('collection_small.csv'))

print(collection['passage'][8770])

collection['passage'] = collection['passage'].str.split()

#print(collection)

list_of_lists_collection = collection['passage'].tolist()

#print(list_of_lists)

tf = []

for doc in list_of_lists_collection:
  tf.append(dict(Counter(doc)))

index, doc_len, avgdl, idf = create_index(list_of_lists_collection)


home / medterms medical dictionary a-z list / intelligence quotient definition Medical Definition of Intelligence quotient Intelligence quotient: An attempt to measure the intelligence of someone. Abbreviated IQ. The IQ score is usually based upon the results of a written test.


In [38]:
# Print the results
print(f"Index: {index}")
print(f"Document Lengths: {doc_len}")
print(f"Average Document Length: {avgdl}")
print(f"Term Frequencies per doc in order: {tf}")
print(f"Inverse Document Frequencies: {idf}")


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# TF percentages rather than numbers, might be useful at some point. NOT USED NOW

In [39]:
def tf_perc(iindex):
    tf = []
    for doc in iindex.values():
        term_freq = {}
        total_words = sum(doc.values())
        for term in doc:
            term_freq[term] = doc[term] / total_words
        tf.append(term_freq)
    return tf

# Example usage
tf2 = tf_perc(index)
print(tf2)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Code below is inspired by: https://medium.com/@evertongomede/understanding-the-bm25-ranking-algorithm-19f6d45c6ce

In [45]:
#BM25 ranking algorithm

def bm25_ranking(tf, idf, docs, query):
    k1 = 1.2
    b = 0.75
    scores = {}

    for doc_id in range(len(docs)):
        score = 0
        doc = docs[doc_id]
        for term in query:
            if str(term) in list(tf[doc_id].keys()):

                score += idf[term] * ((tf[doc_id][term] * (k1 + 1)) / (tf[doc_id][term] + k1 * (1 - b + b * (len(doc) / avgdl))))

        scores[doc_id] = score

    sorted_scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}
    return sorted_scores

# Usage

queries = pd.DataFrame(pd.read_csv('queries_small.csv'))

queries['query'] = queries['query'].str.split()

list_of_lists_query = queries['query'].tolist()

print(list_of_lists_query[0])

scores = []
count = 0
for query in list_of_lists_query:
  count += 1
  print(count)
  scores.append(bm25_ranking(tf, idf, list_of_lists_collection, query))
print(scores[0])


['who', 'published', 'first', 'measure', 'of', 'intelligence']
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
{8770: 17.03425836985787, 7840: 16.901728523361186, 968: 11.869049631462186, 6583: 10.268837785824317, 2836: 9.980594316345698, 4959: 9.868766723437387, 2800: 9.860781047556943, 2118: 9.435650735486277, 6858: 9.339921400223245, 5146: 9.269381108969663, 8242: 8.999200423088562, 5818: 8.908291431958228, 38: 8.72026997032548, 6099: 8.674583217129674, 1745: 8.458259977292387, 8181: 8.43033399680712, 5245: 8.425959074720986, 7465: 8.38543571334718, 3027: 8.340092535502093, 4062: 8.198846800548047, 6987: 8.134569906549814, 8243: 8.126954852975446, 4713: 7.938377198383034, 2901: 7.899996565951297, 4242: 7.850357562409174, 5980: 7.775777288301659, 5829: 7.6509428748129675, 5097: 7.542666697571455, 2314: 7.485184217258495, 3107: 7.430847858019522, 3412: 7.267

In [ ]:
# from relevancy import relevancy_lookup
# import csv

# def process_qrel_file(qrel_path):
#     relevancies = relevancy_lookup()

#     with open(qrel_path) as file:
#         qrel_file = csv.reader(file, delimiter="\t")
#         for line in qrel_file:
#             query, document, relevancy = parse_qrel_line(line)
#             relevancies.add(query, document, relevancy)
#     return relevancies

# def parse_qrel_line(line):
#     #query_id, _, document_id, relevance
#     line = line[0].split()
#     return int(line[0]), line[2], int(line[3])

# qrel_path = "msmarco-docdev-qrels.tsv"
# relevancies = process_qrel_file(qrel_path)
# print(relevancies.relevancies)